In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

In [13]:
# cd /content/drive/MyDrive/alphabetized-th/Notebooks

In [14]:
# !pip install torchtext==0.10.0
# !pip install flair
# !pip install pythainlp

In [15]:
import sys
sys.path.append('../')

In [16]:
from Datasets.ThDatasetVISTEC import ThDatasetVISTEC
# # maindataset = ThDatasetVISTEC("../Data/VISTEC-TP-TH-sample", name="VISTEC-sample")
maindataset = ThDatasetVISTEC("../Data/VISTEC-TP-TH-2021", name="VISTEC")

In [17]:
# !pip install torchtext==0.10.0

In [18]:
from flair.data import Dictionary
from flair.models import LanguageModel
from flair.trainers.language_model_trainer import LanguageModelTrainer, TextCorpus

In [31]:
# make an empty character dictionary
from flair.data import Dictionary
import collections
from tqdm import tqdm
from util import *
import pickle

def build_dictionary(path, dataset):
    char_dictionary = Dictionary()
    counter = collections.Counter()

    with tqdm(total=40000) as pbar:
        tokens = 0
        processed = 0
        for d in dataset.datasets["train"]:
            sent = d["sent"]
            chars = list(sent)
            for i, _ in enumerate(sent):
              chars.append(sent[i:i+2])
              chars.append(sent[i:i+3])

            processed += 1
            tokens += len(chars)
            counter.update(chars)

            pbar.update(1)

    total_count = tokens

    sum = 0
    idx = 0
    
    additionTokens = {}
    nAdd = 0
    
    ppp = []
    for letter, count in counter.most_common():
        sum += count
        percentile = (sum / total_count)

        # comment this line in to use only top X percentile of chars, otherwise filter later        
        if 1-percentile < 0.05: 
          print("Cutoff", percentile)
          break
        
        if len(letter) > 1:
            while True:
              try:
                nAdd += 1
                newletter = chr(0x4E20 + nAdd)
                c = newletter.encode("utf-8")

                if newletter in counter:
                  continue

                break
              except:
                continue
            
            additionTokens[letter] = newletter
            additionTokens[newletter] = letter
            
            letter = newletter
            
            
            
        char_dictionary.add_item(letter)
        idx += 1
    #     print('%d\t%s\t%7d\t%7d\t%f' % (idx, letter, count, sum, percentile))
    
    dict_name = "char_mappings_v_merged"
        
    with open(f'{path}/{dict_name}', 'wb') as f:
        mappings = {
            'idx2item': char_dictionary.idx2item,
            'item2idx': char_dictionary.item2idx
        }
        pickle.dump(mappings, f)
    
    print("Number of sentences", processed)
    print("Number of tokens", tokens)
    print("Number of addition tokens", len(additionTokens.keys()))
    
    return additionTokens

In [32]:
additionTokens = build_dictionary("./Models", maindataset)

 98%|█████████▊| 39000/40000 [00:06<00:00, 5897.03it/s]

Cutoff 0.9500007886228663
Number of sentences 39000
Number of tokens 32588454
Number of addition tokens 27642


In [33]:
with open(f'./Models/merged_addition_tokens', 'wb') as f:
    pickle.dump(additionTokens, f)

In [35]:
with open(f'./Models/merged_addition_tokens', 'rb') as f:
    additionTokens = pickle.load(f)

len(additionTokens.keys())

27642

In [58]:
from flair.data import Dictionary
import collections
from tqdm import tqdm
from util import *
import pickle

def build_dictionary(path, dataset):
    char_dictionary = Dictionary()
    counter = collections.Counter()

    with tqdm(total=40000) as pbar:
        tokens = 0
        processed = 0
        for d in dataset.datasets["train"]:
            sent = d["sent"]
            chars = list(sent)
            for i, _ in enumerate(sent):
              chars.append(sent[i:i+2])
              chars.append(sent[i:i+3])

            processed += 1
            tokens += len(chars)
            counter.update(chars)

            pbar.update(1)

    total_count = tokens

    sum = 0
    idx = 0
    
    additionTokens = {}
    nAdd = 0
    
    ppp = []
    for letter, count in counter.most_common():
        sum += count
        percentile = (sum / total_count)

        # comment this line in to use only top X percentile of chars, otherwise filter later        
        if 1-percentile < 0.01: 
          print("Cutoff", percentile)
          break
        
        if len(letter) > 1:
            while True:
              try:
                nAdd += 1
                newletter = chr(0x4E20 + nAdd)
                c = newletter.encode("utf-8")

                if newletter in counter:
                  continue

                break
              except:
                continue
            
            additionTokens[letter] = newletter
            additionTokens[newletter] = letter
            
            letter = newletter
            
            
            
        char_dictionary.add_item(letter)
        idx += 1
    #     print('%d\t%s\t%7d\t%7d\t%f' % (idx, letter, count, sum, percentile))
    
    dict_name = "char_mappings_v_merged"
        
    with open(f'{path}/{dict_name}', 'wb') as f:
        mappings = {
            'idx2item': char_dictionary.idx2item,
            'item2idx': char_dictionary.item2idx
        }
        pickle.dump(mappings, f)
    
    print("Number of sentences", processed)
    print("Number of tokens", tokens)
    print("Number of addition tokens", len(additionTokens.keys()))
    
    return additionTokens

In [59]:
with open(f'./Models/merged_addition_tokens', 'rb') as f:
    additionTokens = pickle.load(f)

In [60]:
# additionTokens

In [62]:
import random

def transform_merging(sent, p):
    random.seed(42)
    tokens = list(sent)
    
    newtokens = []
    for i, t in enumerate(tokens):
        if random.random() < p and len(newtokens) > 0:
          if len(newtokens[-1]) >= 3:
            # No longer than 3
            newtokens.append(t)
          else:
            newtokens[-1] = newtokens[-1] + t
        else:
          newtokens.append(t)
    
    tokens = newtokens
    newtokens = []
    for t in tokens:
        if len(t) > 1:
            if t in additionTokens:
              newtokens.append(additionTokens[t])
            else:
              newtokens.extend(t)
        else:
            newtokens.append(t)
    return newtokens

transform_merging("แมวกินปลา", p=0.3)
# transform_merging("วจธ", p=1)


['แ', 'ม', 'ว', 'ก', 'ิ', 'น', 'ป', 'ล', 'า']

In [ ]:
from flair.trainers.language_model_trainer import LanguageModelTrainer, TextCorpus, TextDataset
import torch
import logging
log = logging.getLogger("flair")
import random

class MyTextDatasetMerging(TextDataset):
    def __init__(
        self,
        dataset,
        dictionary: Dictionary,
        p_merging: float, 
        expand_vocab: bool = False,
        forward: bool = True,
        split_on_char: bool = True,
        random_case_flip: bool = True,    #ignore
        document_delimiter: str = "\n", #ignore
        shuffle: bool = True,
    ):
        self.dataset = dataset
        self.dictionary = dictionary
        self.split_on_char = split_on_char
        self.forward = forward
        self.random_case_flip = random_case_flip
        self.expand_vocab = expand_vocab
        self.document_delimiter = document_delimiter
        self.shuffle = shuffle
        
        self.p_merging = p_merging

        self.files = []

    def __len__(self):
        return 1

    def __getitem__(self, index=0) -> torch.Tensor:
        """Tokenizes a text file on character basis."""
        lines = []
        for d in self.dataset:
            sent = d["sent"]
            chars = transform_merging(sent, p=self.p_merging)
#             l = chars
            l = chars + [self.document_delimiter]
            lines.append(l)

        log.info(f"read text file with {len(lines)} lines")

        if self.shuffle:
            random.shuffle(lines)
            log.info("shuffled")

        if self.expand_vocab:
            for chars in lines:
                for char in chars:
                    self.dictionary.add_item(char)
                    
        nums = [self.dictionary.get_idx_for_item(char) for chars in lines for char in chars]
        ids = torch.tensor(nums, dtype=torch.long)
        
        if not self.forward:
            ids = ids.flip(0)
        return ids
    
class MyTextCorpus(TextCorpus):
    def __init__(
        self,
        datasets,
        dictionary: Dictionary,
        p_merging: float,
        forward: bool = True,
        character_level: bool = True,
    ):
        self.dictionary: Dictionary = dictionary
        self.forward = forward
        self.split_on_char = character_level
        self.p_merging = p_merging
        
        
        self.random_case_flip = True
        self.expand_vocab = False
        self.document_delimiter = "\n"
        self.shuffle = True
        
        splitmaping = {
            "train": "train",
            "valid": "validation",
            "test": "test",
        }
        
        for k in splitmaping:
            sp = splitmaping[k]
            d = MyTextDatasetMerging(
                datasets.datasets[sp],
                dictionary,
                p_merging = self.p_merging,
                forward = self.forward,
                split_on_char = self.split_on_char,
                expand_vocab = self.expand_vocab,
                random_case_flip = self.random_case_flip,
                document_delimiter=self.document_delimiter,
                shuffle=self.shuffle,
            )
            
            if sp =="train":
                setattr(self, k, d)
            else:
                setattr(self, k, d[0])

In [ ]:
def train_model(maindataset, p_merging, prefix):
    
    dictionary = Dictionary.load_from_file('./Models/char_mappings_v_merged')
    
    print("#Vocab:", len(dictionary.idx2item))
    
    # forward_lm
    corpus = MyTextCorpus(maindataset, dictionary, p_merging=p_merging, forward=True)
    language_model = LanguageModel(dictionary, True, hidden_size=128, nlayers=1)
    trainer = LanguageModelTrainer(language_model, corpus)
    epoch = 20
    model_dir = f'./Models/{prefix}_fwdLM_{p_merging}'
        
    trainer.train(model_dir, sequence_length=280, mini_batch_size=16, max_epochs=epoch)
    
    # backward_lm
    corpus = MyTextCorpus(maindataset, dictionary, p_merging=p_merging, forward=False)
    language_model = LanguageModel(dictionary, False, hidden_size=128, nlayers=1)
    trainer = LanguageModelTrainer(language_model, corpus)
    
    model_dir = f'./Models/{prefix}_bkwLM_{p_merging}'
        
    trainer.train(model_dir, sequence_length=280, mini_batch_size=16, max_epochs=epoch)

In [ ]:
# maindataset = ThDatasetVISTEC("../Data/VISTEC-TP-TH-sample", name="VISTEC-sample")
maindataset = ThDatasetVISTEC("../Data/VISTEC-TP-TH-2021", name="VISTEC")

i=0
for p in range(4, 10):
    train_model(maindataset, p_merging=p/10.0, prefix=f"merging/{(i+1)}")

#Vocab: 152948
2022-05-28 07:54:03,670 read text file with 1000 lines
2022-05-28 07:54:03,673 shuffled
2022-05-28 07:54:05,283 read text file with 10000 lines
2022-05-28 07:54:05,298 shuffled
2022-05-28 07:54:15,384 read text file with 39000 lines
2022-05-28 07:54:15,424 shuffled
2022-05-28 07:54:20,495 Sequence length is 280
2022-05-28 07:54:20,577 Split 1	 - (07:54:20)
2022-05-28 07:54:49,033 | split   1/  1 |   100/ 1523 batches | ms/batch 284.54 | loss 8.3868 | ppl 4388.6357
2022-05-28 07:55:17,449 | split   1/  1 |   200/ 1523 batches | ms/batch 284.11 | loss 7.3356 | ppl 1533.9975
2022-05-28 07:55:46,050 | split   1/  1 |   300/ 1523 batches | ms/batch 285.95 | loss 7.0196 | ppl 1118.3681
2022-05-28 07:56:15,014 | split   1/  1 |   400/ 1523 batches | ms/batch 289.57 | loss 6.6805 | ppl 796.6779
2022-05-28 07:56:44,444 | split   1/  1 |   500/ 1523 batches | ms/batch 294.28 | loss 6.3908 | ppl 596.3095
2022-05-28 07:57:14,156 | split   1/  1 |   600/ 1523 batches | ms/batch 297.0

In [ ]:
# maindataset = ThDatasetVISTEC("../Data/VISTEC-TP-TH-sample", name="VISTEC-sample")
maindataset = ThDatasetVISTEC("../Data/VISTEC-TP-TH-2021", name="VISTEC")

for i in range(2, 5):
    for p in range(1, 10):
        train_model(maindataset, p_merging=p/10.0, prefix=f"merging/{(i+1)}")

In [ ]:
 "DONE"